In [1]:
%%capture
!pip install --upgrade torch torchvision torchaudio
!pip install pytorch_lightning
!pip install wandb
!pip install gdown

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/10UJTh0YUpVk75H2KMIiZ61sDsC-woWl0/view?usp=sharing


Downloading...
From (original): https://drive.google.com/uc?id=10UJTh0YUpVk75H2KMIiZ61sDsC-woWl0
From (redirected): https://drive.google.com/uc?id=10UJTh0YUpVk75H2KMIiZ61sDsC-woWl0&confirm=t&uuid=e318ac54-f227-4e11-a82d-170eafcbe77d
To: /content/temporal_ds.zip
100% 1.69G/1.69G [00:38<00:00, 44.4MB/s]


In [3]:
%%capture
!unzip temporal_ds.zip

In [5]:
# Define the transformations
resize = transforms.Resize((112, 112))
horizontal_flip = transforms.RandomHorizontalFlip(p=0.5)
rotation = transforms.RandomRotation(degrees=10)
color_jitter = transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
random_crop = transforms.RandomResizedCrop(size=(112, 112), scale=(0.9, 1.0), ratio=(0.9, 1.1))
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])



# Function to apply the transformations using the generated parameters
def apply_transform_list(imgs):
    # Seed the random number generators
    seed = np.random.randint(2147483647)
    random.seed(seed)
    torch.manual_seed(seed)

    # Generate random transformation parameters
    params = {
        'horizontal_flip': random.random(),
        'rotation': random.uniform(-10, 10),
        'brightness': random.uniform(0.9, 1.1),
        'contrast': random.uniform(0.9, 1.1),
        'saturation': random.uniform(0.9, 1.1),
        'hue': random.uniform(-0.1, 0.1),
        'crop_params': random_crop.get_params(resize(imgs[0]), scale=(0.9, 1.0), ratio=(0.9, 1.1))
    }

    new_imgs = []

    for img in imgs:
        img = resize(img)

        if params['horizontal_flip'] < 0.5:
            img = transforms.functional.hflip(img)

        img = transforms.functional.rotate(img, params['rotation'])

        img = transforms.functional.adjust_brightness(img, params['brightness'])
        img = transforms.functional.adjust_contrast(img, params['contrast'])
        img = transforms.functional.adjust_saturation(img, params['saturation'])
        img = transforms.functional.adjust_hue(img, params['hue'])
        img = transforms.functional.resized_crop(img, *params['crop_params'], size=(112, 112))
        img = to_tensor(img)
        img = normalize(img)

        new_imgs.append(img)

    return new_imgs

In [6]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import glob
import random
import numpy as np
from torchmetrics import Accuracy, Precision, Recall
import wandb
from pytorch_lightning.loggers import WandbLogger
import torchvision.transforms as T

In [7]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

In [24]:


class FireSeriesDataset(Dataset):
    def __init__(self, root_dir, img_size=112, transform=None):
        self.transform = transform
        self.sets = glob.glob(f"{root_dir}/**/*")
        self.img_size=img_size
        random.shuffle(self.sets)

    def __len__(self):
        return len(self.sets)

    def __getitem__(self, idx):
        img_folder = self.sets[idx]
        img_list = glob.glob(f"{img_folder}/*.jpg")

        labels = []
        for file in img_list:
            label_file = file.replace("images", "labels").replace(".jpg", ".txt")
            with open(label_file, "r") as f:
                lines = f.readlines()

            labels.append(np.array(lines[0].split(" ")[1:5]).astype("float"))

        labels = np.array(labels)
        xc = np.median(labels[:, 0])
        yc = np.median(labels[:, 1])
        wb = np.max(labels[:, 2])
        hb = np.max(labels[:, 3])

        # Load all images first
        images = [Image.open(file) for file in img_list]
        w, h = images[0].size

        crop_size = max(wb*h, hb*h)
        if crop_size < self.img_size:
            crop_size = self.img_size

        x0 = int(xc * w - crop_size / 2)
        y0 = int(yc * h - crop_size / 2)
        x1 = int(xc * w  + crop_size / 2)
        y1 = int(yc * h + crop_size / 2)

        img_list = []

        for im in images:
            cropped_image = im.crop(
                (x0, y0, x1,y1))

            cropped_image = cropped_image.resize((self.img_size, self.img_size))
            img_list.append(cropped_image)

        tensor_list = apply_transform_list(img_list)
        # txc, w,h to t, x,c,w,h
        tensor_list = [tensor.unsqueeze(0) for tensor in tensor_list]

        # Concatena a lo largo de una nueva dimensión al principio, formando un tensor de (T, C, W, H)
        combined_tensor = torch.cat(tensor_list, dim=0)




        return torch.cat(tensor_list), int(img_folder.split("/")[-2])


In [25]:
train = FireSeriesDataset(root_dir = "temporal_ds/images/train")

In [28]:

class FireDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=16, img_size=112, num_workers=12):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_workers = num_workers



    def setup(self, stage=None):
        self.train_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "train"), self.img_size
        )
        self.val_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "val"), self.img_size
        )

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)


In [57]:

class FireClassifier(pl.LightningModule):
    def __init__(self, learning_rate=1e-4):
        super(FireClassifier, self).__init__()
        self.save_hyperparameters()

        # Usamos una ResNet como extractor de características.
        # Pretrained sobre ImageNet, usualmente se carga con 3 canales.
        resnet = models.resnet50(pretrained=True)
        # Removemos la capa final para usarla como extractor de características.
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-2])

        # LSTM que procesará las características extraídas.
        # Número de características de la salida del último bloque conv de ResNet.
        num_features = resnet.fc.in_features
        self.lstm = nn.LSTM(input_size=32768, hidden_size=256, batch_first=True, num_layers=1)

        # Capa de clasificación.
        self.classifier = nn.Linear(256, 1)  # Salida binaria

        # Dropout para regularización
        self.dropout = nn.Dropout(0.2)

        # Métricas
        self.train_accuracy = Accuracy(task="binary")
        self.val_accuracy = Accuracy(task="binary")
        self.train_precision = Precision(task="binary")
        self.val_precision = Precision(task="binary")
        self.train_recall = Recall(task="binary")
        self.val_recall = Recall(task="binary")

    def forward(self, x):
        # x shape: [batch_size, seq_length, channels, height, width]
        # Procesa cada imagen de la secuencia a través del extractor de características.
        batch_size, seq_length, C, H, W = x.size()
        x = x.view(batch_size * seq_length, C, H, W)
        x = self.feature_extractor(x)

        # Reformatear salida para la LSTM
        # Deberás asegurarte que las dimensiones coincidan con lo que espera la LSTM.
        x = x.view(batch_size, seq_length, -1)

        # Pasar las características por la LSTM
        x, _ = self.lstm(x)


        # Solo nos interesa la última salida de la secuencia para la clasificación
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch

        y_hat = self(x).squeeze()

        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        acc = self.train_accuracy(torch.sigmoid(y_hat), y.int())
        precision = self.train_precision(torch.sigmoid(y_hat), y.int())
        recall = self.train_recall(torch.sigmoid(y_hat), y.int())
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        self.log("train_precision", precision)
        self.log("train_recall", recall)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()


        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        acc = self.val_accuracy(torch.sigmoid(y_hat), y.int())
        precision = self.val_precision(torch.sigmoid(y_hat), y.int())
        recall = self.val_recall(torch.sigmoid(y_hat), y.int())
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        self.log("val_precision", precision)
        self.log("val_recall", recall)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams['learning_rate'], weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler
        }


In [58]:

# Initialize the DataModule
data_dir = "temporal_ds/images"
data_module = FireDataModule(data_dir)

# Initialize the model
model = FireClassifier()

# Define callbacks
checkpoint_callback = ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1)

# Initialize WandbLogger
wandb_logger = WandbLogger(project='fire_detection_project')

# Initialize the Trainer
trainer = pl.Trainer(
    max_epochs=50,
    callbacks=[checkpoint_callback],
    logger=wandb_logger
)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
# Train the model
trainer.fit(model, data_module)
wandb.finish()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | feature_extractor | Sequential      | 23.5 M | train
1 | lstm              | LSTM            | 33.8 M | train
2 | classifier        | Linear          | 257    | train
3 | dropout           | Dropout         | 0      | train
4 | train_accuracy    | BinaryAccuracy  | 0      | train
5 | val_accuracy      | BinaryAccuracy  | 0      | train
6 | train_precision   | BinaryPrecision | 0      | train
7 | val_precision     | BinaryPrecision | 0      | train
8 | train_recall      | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
wandb.finish()